In [47]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.utils.multiclass import type_of_target
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

In [31]:
IS_LOCAL = True
if(IS_LOCAL):
    PATH="./"
else:
    PATH="../input/"
os.listdir(PATH)

['lgbm_importances.png',
 'Keras+LGBM  Feature Eng+Sampling Prediction.ipynb',
 'combination_target_test.png',
 'submission_where.csv',
 'For submission.ipynb',
 'test.csv',
 'submission.csv',
 'submission2.csv',
 'FI.png',
 'Santander EDA and Prediction.ipynb',
 'submission_sampled_agmted_agmted_agmted.csv',
 'comb_submission.csv',
 'train.csv',
 '.ipynb_checkpoints',
 'combination_val.png',
 'LGB 2 leaves + augment.ipynb',
 'sample_submission.csv']

In [32]:
train_df = pd.read_csv(PATH+"train.csv")
test_df = pd.read_csv(PATH+"test.csv")

In [33]:
idx = features = train_df.columns.values[2:202]
for df in [test_df, train_df]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

In [34]:
print('Train and test columns: {} {}'.format(len(train_df.columns), len(test_df.columns)))

Train and test columns: 210 209


In [35]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [36]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

In [51]:
type_of_target(target)

'binary'

In [53]:
target.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    pass

In [ ]:
# folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
folds = StratifiedKFold(n_splits=2, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

#     num_round = 1000000
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [ ]:
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission2.csv", index=False)

In [ ]:
sub_df.head()

In [ ]:
sub_copy = sub_df.copy()

In [ ]:
mask = sub_copy.target > 0.5
column_name = 'target'
# sub_copy.loc[mask, column_name] = 1
# sub_copy.loc[!(mask), column_name] = 0

sub_copy[column_name] = np.where(sub_copy[column_name]>0.8, 1, 0)

In [ ]:
sub_copy.head()

In [ ]:
sub_copy.to_csv("submission_where.csv", index=False)